## 3. Measuring Zero-Shot MATH Performance

In [ ]:
from vllm import LLM, SamplingParams

llm = LLM(model="Qwen/Qwen2.5-Math-1.5B")

In [ ]:
# Sample prompts.
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

# Create a sampling params object, stopping generation on newline.
sampling_params = SamplingParams(
    temperature=1.0, top_p=1.0, max_tokens=1024, stop=["\n"]
)

# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
outputs = llm.generate(prompts, sampling_params)

In [ ]:
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

In [ ]:
# https://huggingface.co/datasets/EleutherAI/hendrycks_math

import pandas as pd

# All MATH dataset subjects
subjects = [
    "algebra", "counting_and_probability", "geometry", 
    "intermediate_algebra", "number_theory", "prealgebra", "precalculus"
]

# Load train data from all subjects
train_dfs = []
for subject in subjects:
    df = pd.read_parquet(f"hf://datasets/EleutherAI/hendrycks_math/{subject}/train-00000-of-00001.parquet")
    df['subject'] = subject
    train_dfs.append(df)

df_train = pd.concat(train_dfs, ignore_index=True)
print(f"Total train examples: {len(df_train)}")

# Load test data from all subjects
test_dfs = []
for subject in subjects:
    df = pd.read_parquet(f"hf://datasets/EleutherAI/hendrycks_math/{subject}/test-00000-of-00001.parquet")
    df['subject'] = subject
    test_dfs.append(df)

df_test = pd.concat(test_dfs, ignore_index=True)
print(f"Total test examples: {len(df_test)}")
# print(f"Columns: {df_test.columns.tolist()}")

In [ ]:
from vllm import LLM, SamplingParams
from typing import Callable, List
from cs336_alignment.drgrpo_grader import r1_zero_reward_fn
import json
import pandas as pd

def get_prompts_and_answers(prompt_template, df):
    prompts = [prompt_template.replace("{question}", p) for p in df.problem.tolist()]
    return prompts, df.answer.tolist()

def evaluate_vllm(
    vllm_model: LLM,
    eval_sampling_params: SamplingParams,
    prompts: List[str],
    reward_fn: Callable[[str, str], dict[str, float]],
    solutions: List[str],
    output_file: str = "eval_results.jsonl"
) -> None:
    """
    Evaluatea languagemodelon a listof prompts,
    compute evaluation metrics, and serialize results to disk.
    """
    responses = vllm_model.generate(prompts, eval_sampling_params)
    solutions_generated = [opt.outputs[0].text for opt in responses]

    evals = [reward_fn(sol_gen, sol) for sol_gen, sol in zip(solutions_generated, solutions)]

    # Serialize the prompts, solutions, solutions generated, and corresponding evals to disk
    with open(output_file, 'w') as f:
        for prompt, solution, sol_gen, eval_dict in zip(prompts, solutions, solutions_generated, evals):
            result = {
                "prompt": prompt,
                "ground_truth": solution,
                "generated": sol_gen,
                "eval": eval_dict
            }
            f.write(json.dumps(result) + '\n')

    return evals, solutions_generated

In [ ]:
llm = LLM(model="Qwen/Qwen2.5-Math-1.5B")

prompt_r1_zero_file = "/home/azureuser/localfiles/cs336-assignment5-alignment-mine/cs336_alignment/prompts/r1_zero.prompt"
with open(prompt_r1_zero_file) as f:
    prompt_r1_zero = f.read()

df = pd.read_json("/home/azureuser/localfiles/cs336-assignment5-alignment-mine/data/validation.jsonl", lines=True)

In [ ]:
df.head()

In [ ]:
# Create a sampling params object, stopping generation on newline.
sampling_params = SamplingParams(
    temperature=1.0, top_p=1.0, max_tokens=1024, stop=["</answer>"],
    include_stop_str_in_output=True,
)

prompts, answers = get_prompts_and_answers(prompt_r1_zero, df)
evals, answers_generated = evaluate_vllm(
    llm, sampling_params, prompts, r1_zero_reward_fn, answers)

## 3.1 Using vLLM for offline language model inference
### math_baseline
1. Done
1. Commentary on model and reward func perf
    1. See cell below for distribution. 
    1. For cases with zero format reward: most are because of the model failed to generate the answer tags or not in the right format. For cases with non-zero format reward but zero answer reward: 50/50 of wrong answer and parser failure
1. less than 3% get both format and answer rewards

In [ ]:
# math_baseline.2
df_eval = pd.DataFrame(evals)
# Check rows where format_reward is 1 and answer_reward is 1
print(f"Format reward 1, Answer reward 1: {((df_eval["format_reward"] == 1) & (df_eval["answer_reward"] == 1)).sum()}")
print(f"Format reward 1, Answer reward 0: {((df_eval["format_reward"] == 1) & (df_eval["answer_reward"] == 0)).sum()}")
print(f"Format reward 0, Answer reward 0: {((df_eval["format_reward"] == 0) & (df_eval["answer_reward"] == 0)).sum()}")

In [ ]:
counts_by_category = df.groupby("subject")["problem"].count()
accurate_by_category = df[df_eval.reward == 1].groupby("subject")["problem"].count()
accuracy_percentage = (accurate_by_category / counts_by_category) * 100
accuracy_percentage

In [ ]:
sample_ids = df_eval[(df_eval.format_reward==1) & (df_eval.answer_reward==0)].sample(10).index
sample_problems = (df["problem"].tolist()[i] for i in sample_ids)
sample_answers = (df["answer"].tolist()[i] for i in sample_ids)
sample_answers_generated = (answers_generated[i] for i in sample_ids)

In [ ]:
print(next(sample_problems))
print()
print(next(sample_answers))
print()
print(next(sample_answers_generated))


## 4. Supervised Finetuning for MATH

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
import torch
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-Math-1.5B",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
).to('cuda')
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Math-1.5B")

### 4.2 SFT helper methods

In [28]:
import torch
from transformers import AutoTokenizer, PreTrainedTokenizer
import torch.nn.functional as F

def tokenize_prompt_and_output(
        prompt_strs: list[str],
        output_strs: list[str],
        tokenizer: PreTrainedTokenizer,
) -> dict[str, torch.Tensor]:
    max_seqlen = 0
    tokens_list = []
    masks = []
    for prompt_str, output_str in zip(prompt_strs, output_strs):
        tokens_prompt = tokenizer.encode(prompt_str) 
        tokens_output = tokenizer.encode(output_str)
        tokens = tokens_prompt + tokens_output
        mask = [False] * len(tokens_prompt) + [True] * len(tokens_output)
        tokens_list.append(tokens)
        masks.append(mask)

        if len(tokens) > max_seqlen:
            max_seqlen = len(tokens)
    
    tokens_padded = [tokens + [tokenizer.pad_token_id] * (max_seqlen - len(tokens)) for tokens in tokens_list]
    masks_padded = [mask + [False] * (max_seqlen - len(mask)) for mask in masks]

    result = {
        "input_ids": torch.tensor(tokens_padded)[:, :-1],
        "labels": torch.tensor(tokens_padded)[:, 1:],
        "response_mask": torch.tensor(masks_padded)[:, 1:],
    }
    return result

In [31]:
prompt_strs = ['Hello, world!', 'This is a test.', 'This is another test.']
output_strs = ['Hello, world!', 'This is a test.', 'This is another test.']

tokenize_prompt_and_output(prompt_strs, output_strs, tokenizer)

{'input_ids': tensor([[  9707,     11,   1879,      0,   9707,     11,   1879,      0, 151643],
         [  1986,    374,    264,   1273,     13,   1986,    374,    264,   1273],
         [  1986,    374,   2441,   1273,     13,   1986,    374,   2441,   1273]]),
 'labels': tensor([[    11,   1879,      0,   9707,     11,   1879,      0, 151643, 151643],
         [   374,    264,   1273,     13,   1986,    374,    264,   1273,     13],
         [   374,   2441,   1273,     13,   1986,    374,   2441,   1273,     13]]),
 'response_mask': tensor([[False, False, False,  True,  True,  True,  True, False, False],
         [False, False, False, False,  True,  True,  True,  True,  True],
         [False, False, False, False,  True,  True,  True,  True,  True]])}

In [11]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Math-1.5B")

In [24]:
tokenizer.encode("eating", return_tensors='pt')

tensor([[90855]])

In [27]:
tokenizer.pad_token_id

151643

In [15]:
torch.tensor([[1, 2], [3, 4]])

tensor([[1, 2],
        [3, 4]])

In [2]:
import numpy as np 

data = np.load("/home/azureuser/localfiles/cs336-assignment5-alignment-mine/tests/_snapshots/test_tokenize_prompt_and_output.npz")

In [5]:
# See what arrays are in the file
print("Available arrays:", data.files)

# Print each array's shape and content
for key in data.files:
    print(f"\n{key}:")
    print(f"  Shape: {data[key].shape}")
    print(f"  Content: {data[key]}")

Available arrays: ['input_ids', 'labels', 'response_mask']

input_ids:
  Shape: (3, 9)
  Content: [[  9707     11   1879      0   9707     11   1879      0 151643]
 [  1986    374    264   1273     13   1986    374    264   1273]
 [  1986    374   2441   1273     13   1986    374   2441   1273]]

labels:
  Shape: (3, 9)
  Content: [[    11   1879      0   9707     11   1879      0 151643 151643]
 [   374    264   1273     13   1986    374    264   1273     13]
 [   374   2441   1273     13   1986    374   2441   1273     13]]

response_mask:
  Shape: (3, 9)
  Content: [[False False False  True  True  True  True False False]
 [False False False False  True  True  True  True  True]
 [False False False False  True  True  True  True  True]]
